In [22]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import urllib
import sys
import time
import pytesseract
import time
import numpy as np
import cv2
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

from PIL import Image
from pytesseract import image_to_string
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException 
from PIL import Image, ImageFilter
from datetime import datetime
from selenium.webdriver.firefox.options import Options

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
URL_SCRAPING = 'https://www.sunarp.gob.pe/seccion/servicios/detalles/0/c3.html'
GECKO_PATH = 'geckodriver.exe'
FIREFOX_PATH = os.environ['FIREFOX_HOME']
CAPTCHA_PATH = 'img/captchas'

binary = FirefoxBinary(FIREFOX_PATH + r'\firefox.exe')

profile = webdriver.FirefoxProfile()
profile.set_preference("dom.disable_beforeunload", True)
options = Options()
options.add_argument("--headless")

profile.set_preference("browser.tabs.remote.autostart", False)
profile.set_preference("browser.tabs.remote.autostart.1", False)
profile.set_preference("browser.tabs.remote.autostart.2", False)
profile.set_preference("browser.tabs.remote.force-enable", False)

In [24]:
driver = instance_driver(options, profile, binary, GECKO_PATH)
load_page(driver, URL_SCRAPING)
navigate_to_form(driver, xpath = '//div//a')

In [25]:
captcha_element = driver.find_element_by_id("ctl00_MainContent_captch_cv")

In [34]:
get_captcha(driver, captcha_element, f"{os.getcwd()}\\img\\captchas")

C:\Users\S82483\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:1019: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  "type. It should end with a `.png` extension", UserWarning)


PermissionError: [Errno 13] Permission denied: 'D:\\Datos de Usuarios\\S82483\\My Documents\\projects\\opencv-tutorials\\src\\notebooks\\img\\captchas'